test to parse a small dataset from doctorslounge:
> score_Eye diseases (Ophthalmology).xlsx

structure: two tables
- text
    - text_id
    - type {answser, question} (maybe not needed)
    - user_id (refers to user table id)
- user
    - user_id
    - type {patient, doctor, nurse}
    - age
    - gender

---

In [1]:
import pandas as pd

---

#### load data

In [72]:
df = pd.read_excel('data/score_Eye diseases (Ophthalmology).xlsx', sheet_name=0)
df.head(2)

,Disease,Topic,Url,Question,Answer,Asker ID,Asker Gender,Asker Dummy,Asker Age,Answerer Title,...,Question Time Stamp,Answer Time Stamp,Question Score,Answer Score,Patient Sentiment,Doctor Sentiment,Patient Topic,Doctor Topic,Patient Formality,Doctor Formality
0,Eye Diseases,Flashing Lights. -,https://www.doctorslounge.com/forums/./viewtop...,I've been seeing flashing lights now for about...,"Hello,While retinal detachments remain the mos...",Rene√©,Female,1,27.0,Dr.M.jagesh kamath,...,"Tue Nov 10, 2009 2:00 am","Wed Sep 15, 2010 11:05 am",0.87,0.91,0.25,0.57,0.76,1.00,0.00,0.95
1,Eye Diseases,Large optic nerves suspicious for glaucoma -,https://www.doctorslounge.com/forums/./viewtop...,How common are large optic nerve discs that ar...,"Hi, You have not mentioned her disc size. Howe...",Ruthiejb,Female,1,NaN,Dr.M.Aroon kamath,...,"Sat Jul 31, 2010 8:57 am","Fri Aug 20, 2010 1:52 am",0.74,0.98,0.15,0.51,0.79,0.99,0.16,1.00


In [73]:
df.columns

Index(['Disease', 'Topic', 'Url', 'Question', 'Answer', 'Asker ID',
       'Asker Gender', 'Asker Dummy', 'Asker Age', 'Answerer Title',
       'Answerer ID', 'Answerer Gender', 'Answerer Dummy', 'Answerer Age',
       'Question Time Stamp', 'Answer Time Stamp', 'Question Score',
       'Answer Score', 'Patient Sentiment', 'Doctor Sentiment',
       'Patient Topic ', 'Doctor Topic', 'Patient Formality',
       'Doctor Formality'],
      dtype='object')

extract relevant column list

In [74]:
col_list = ['Question', 'Answer', 
            'Asker ID', 'Asker Gender', 'Asker Age',
            'Answerer ID', 'Answerer Title',
            'Answerer Gender', 'Answerer Age']
df = df[col_list]
df.head(2)

,Question,Answer,Asker ID,Asker Gender,Asker Age,Answerer ID,Answerer Title,Answerer Gender,Answerer Age
0,I've been seeing flashing lights now for about...,"Hello,While retinal detachments remain the mos...",Rene√©,Female,27.0,Medical Doctor,Dr.M.jagesh kamath,Male,68.0
1,How common are large optic nerve discs that ar...,"Hi, You have not mentioned her disc size. Howe...",Ruthiejb,Female,NaN,Medical Doctor,Dr.M.Aroon kamath,Male,69.0


rename

In [75]:
df.columns = ['question', 'answer', 
              'asker_id', 'asker_gender', 'asker_age',
              'answerer_id', 'answerer_title', 'answerer_gender', 'answerer_age'
             ]

In [76]:
df.head(2)

,question,answer,asker_id,asker_gender,asker_age,answerer_id,answerer_title,answerer_gender,answerer_age
0,I've been seeing flashing lights now for about...,"Hello,While retinal detachments remain the mos...",Rene√©,Female,27.0,Medical Doctor,Dr.M.jagesh kamath,Male,68.0
1,How common are large optic nerve discs that ar...,"Hi, You have not mentioned her disc size. Howe...",Ruthiejb,Female,NaN,Medical Doctor,Dr.M.Aroon kamath,Male,69.0


both patients and doctors/nurse. strip string for more coherent results

In [77]:
df.asker_id = df.asker_id.str.strip().values
df.answerer_id = df.answerer_id.str.strip().values

add pair id to match question and answer (maybe not needed but let's see)

In [78]:
df['qa_id'] = pd.np.arange(df.shape[0])

---

#### user table

##### patients

In [79]:
patient_df = df[['asker_id', 'asker_age', 'asker_gender']]
patient_df.head(2)

,asker_id,asker_age,asker_gender
0,Rene√©,27.0,Female
1,Ruthiejb,NaN,Female


add role

In [80]:
patient_df.loc[:, 'role'] = 'patient'

/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


change column names

In [81]:
patient_df.columns

Index(['asker_id', 'asker_age', 'asker_gender', 'role'], dtype='object')

In [82]:
patient_df.columns = ['user_id', 'age', 'gender', 'role']
patient_df.head(2)

,user_id,age,gender,role
0,Rene√©,27.0,Female,patient
1,Ruthiejb,NaN,Female,patient


##### doctors

In [83]:
doctor_df = df[['answerer_id', 'answerer_title', 'answerer_age', 'answerer_gender']]
doctor_df.head(2)

,answerer_id,answerer_title,answerer_age,answerer_gender
0,Medical Doctor,Dr.M.jagesh kamath,68.0,Male
1,Medical Doctor,Dr.M.Aroon kamath,69.0,Male


update columns

In [84]:
doctor_df.columns = ['role', 'user_id', 'age', 'gender']
doctor_df.head(2)

,role,user_id,age,gender
0,Medical Doctor,Dr.M.jagesh kamath,68.0,Male
1,Medical Doctor,Dr.M.Aroon kamath,69.0,Male


##### append

In [85]:
user_df = doctor_df.append(patient_df, ignore_index=True, sort=True)

add unique id

In [86]:
user_df['id'] = pd.np.arange(user_df.shape[0])

In [87]:
user_df.head(2)

,age,gender,role,user_id,id
0,68.0,Male,Medical Doctor,Dr.M.jagesh kamath,0
1,69.0,Male,Medical Doctor,Dr.M.Aroon kamath,1


##### check

In [88]:
user_df.role.value_counts()

patient            106
Medical Doctor      81
Nurse Assistant      7
Nurse                4
Name: role, dtype: int64

In [89]:
user_df.dtypes

age        float64
gender      object
role        object
user_id     object
id           int64
dtype: object

In [90]:
user_df.gender.value_counts()

Male      140
Female     57
Name: gender, dtype: int64

In [91]:
user_df.age.values

array([68., 69., 68., nan, nan, nan, 69., 69., 69., 69., 69., 69., 69.,
       68., nan, nan, nan, 68., 49., 49., 49., 49., 49., 49., 49., 69.,
       69., 69., 69., nan, nan, nan, nan, nan, 49., 54., 49., 49., 49.,
       49., 49., 49., 49., 49., 49., 49., 49., 49., 73., 49., 49., 49.,
       49., 49., 49., 49., 49., 49., 49., 49., 49., 49., 49., 49., 49.,
       49., 49., 49., 49., 49., 49., 49., 49., 49., 49., 49., 49., 49.,
       49., 49., 73., 73., 49., 49., 49., 49., 49., 49., 49., 49., 49.,
       49., 49., 49., 49., 73., 73., 73., nan, nan, 62., nan, 73., 62.,
       62., nan, 27., nan, 36., 31., 53., 24., 27., 28., 46., 34., 34.,
       27., 28., 37., 57., 28., 23., 41., 36., 58., 33., 58., 58., 55.,
       68., 58., 38., 33., 43., 31., 33., 36., 29., 30., 26., 58., 29.,
       51., 33., 27., 46., 37., nan, 28., nan, nan, 33., 34., 34., 35.,
       48., 27., 60., 33., 28., 44., 51., 35., 35., 31., 44., 26., 35.,
       36., 56., 44., 46., 28., 29., 31., 44., 54., 55., 26., 41

---

#### text table

##### question

In [92]:
question_df = df[['qa_id', 'question', 'asker_id', 'answerer_title']]
question_df

,qa_id,question,asker_id,answerer_title
0,0,I've been seeing flashing lights now for about...,Rene√©,Dr.M.jagesh kamath
1,1,How common are large optic nerve discs that ar...,Ruthiejb,Dr.M.Aroon kamath
2,2,My husband has no previous issues with his eye...,kkb,Dr.M.jagesh kamath
3,3,I had an eye injury a few years back and the d...,chevyboy8602,NaN
4,4,Had a corneal graft done in 1996. Vision was ...,buzzac,NaN
5,5,Hi-My cornea got scratched and I think it got ...,Gbiscuit,NaN
6,6,"Just moments ago, I experienced sudden blurred...",N_eight,Dr.M.Aroon kamath
7,7,I'm not sure if I'm posting this in the right ...,jmoney,Dr.M.Aroon kamath
8,8,"Hello sir/mam, My son aged 4 years is known to...",mscn71,Dr.M.Aroon kamath
9,9,Hello Doctor I am 26 years old. Since 3 days ...,jinkle007,Dr.M.Aroon kamath


rename columns

In [93]:
question_df.columns = ['qa_id', 'text', 'asker_id', 'answerer_id']

add type

In [101]:
question_df['type'] = 'q'

/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


##### answer

In [95]:
answer_df = df[['qa_id', 'answer', 'asker_id', 'answerer_title']]
answer_df

,qa_id,answer,asker_id,answerer_title
0,0,"Hello,While retinal detachments remain the mos...",Rene√©,Dr.M.jagesh kamath
1,1,"Hi, You have not mentioned her disc size. Howe...",Ruthiejb,Dr.M.Aroon kamath
2,2,"Hello,Pyogenic granulomas can occur on the ski...",kkb,Dr.M.jagesh kamath
3,3,NaN,chevyboy8602,NaN
4,4,NaN,buzzac,NaN
5,5,NaN,Gbiscuit,NaN
6,6,"Hi, At the outset, i must stress that a consul...",N_eight,Dr.M.Aroon kamath
7,7,"Hi, Your description strongly resembles the si...",jmoney,Dr.M.Aroon kamath
8,8,"Hi, From the history provided, some possibilit...",mscn71,Dr.M.Aroon kamath
9,9,"Hi, You seem to have predominantly 'tenderness...",jinkle007,Dr.M.Aroon kamath


rename columns

In [98]:
answer_df.columns = ['qa_id', 'text', 'asker_id', 'answerer_id']

add type

In [100]:
answer_df['type'] = 'a'

/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
answer_df.head(2)

,qa_id,text,asker_id,answerer_id,type
0,0,"Hello,While retinal detachments remain the mos...",Rene√©,Dr.M.jagesh kamath,a
1,1,"Hi, You have not mentioned her disc size. Howe...",Ruthiejb,Dr.M.Aroon kamath,a


##### append

In [103]:
text_df = question_df.append(answer_df, ignore_index=True)
text_df.head(2)

,qa_id,text,asker_id,answerer_id,type
0,0,I've been seeing flashing lights now for about...,Rene√©,Dr.M.jagesh kamath,q
1,1,How common are large optic nerve discs that ar...,Ruthiejb,Dr.M.Aroon kamath,q


drop if `text` field is missing

In [105]:
text_df.dropna(subset=['text'], axis=0, inplace=True)
text_df.shape

(198, 5)

In [106]:
text_df['type'].value_counts()

q    106
a     92
Name: type, dtype: int64

---

#### wrapper

add disease type to `text` table

In [8]:
def wrapper(f):
    df = pd.read_excel(f, sheet_name=0)
    ## retrieve the first disease name
    assert pd.np.all(df['Disease'].values == df.loc[0, 'Disease'])
    disease_name = df.loc[0, 'Disease']
    ## extract relevant column list
    col_list = ['Question', 'Answer', 
                'Asker ID', 'Asker Gender', 'Asker Age',
                'Answerer ID', 'Answerer Title',
                'Answerer Gender', 'Answerer Age']
    df = df[col_list]
    
    ## rename
    df.columns = ['question', 'answer',
                  'asker_id', 'asker_gender', 'asker_age',
                  'answerer_id', 'answerer_title', 'answerer_gender', 'answerer_age'
                 ]
    ## both patients and doctors/nurse. strip string for more coherent results
    df.asker_id = df.asker_id.str.strip().values
    df.answerer_id = df.answerer_id.str.strip().values

    ## add pair id to match question and answer (maybe not needed but let's see)
    df['qa_id'] = pd.np.arange(df.shape[0])

    #### user table
    ##### patients
    patient_df = df[['asker_id', 'asker_age', 'asker_gender']]
    ## add role
    patient_df.loc[:, 'role'] = 'patient'
    ## change column names
    patient_df.columns = ['user_id', 'age', 'gender', 'role']
    ##### doctors
    doctor_df = df[['answerer_id', 'answerer_title', 'answerer_age', 'answerer_gender']]
    ## update columns
    doctor_df.columns = ['role', 'user_id', 'age', 'gender']
    
    ##### append
    user_df = doctor_df.append(patient_df, ignore_index=True, sort=True)
    user_df['id'] = pd.np.arange(user_df.shape[0])

    #### text table
    ##### question
    question_df = df[['qa_id', 'question', 'asker_id', 'answerer_title']]
    ## rename columns
    question_df.columns = ['qa_id', 'text', 'asker_id', 'answerer_id']
    ## add type
    question_df['type'] = 'q'
    ##### answer
    answer_df = df[['qa_id', 'answer', 'asker_id', 'answerer_title']]
    ## rename columns
    answer_df.columns = ['qa_id', 'text', 'asker_id', 'answerer_id']
    ## add type
    answer_df['type'] = 'a'

    ##### append
    text_df = question_df.append(answer_df, ignore_index=True)
    ## add disease name
    text_df['disease'] = disease_name
    ## drop if `text` field is missing
    text_df.dropna(subset=['text'], axis=0, inplace=True)
    
    return user_df, text_df

##### text it

In [9]:
f = 'data/score_Eye diseases (Ophthalmology).xlsx'

In [10]:
user_df, text_df = wrapper(f)

/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/zhiyzuo/.virtualenvs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [11]:
user_df.shape

(212, 5)

In [12]:
user_df.head(2)

,age,gender,role,user_id,id
0,68.0,Male,Medical Doctor,Dr.M.jagesh kamath,0
1,69.0,Male,Medical Doctor,Dr.M.Aroon kamath,1


In [13]:
text_df.head(2)

,qa_id,text,asker_id,answerer_id,type,disease
0,0,I've been seeing flashing lights now for about...,Rene√©,Dr.M.jagesh kamath,q,Eye Diseases
1,1,How common are large optic nerve discs that ar...,Ruthiejb,Dr.M.Aroon kamath,q,Eye Diseases


In [14]:
text_df.to_csv('test-data/test.csv', index=False)

---

now turn everything to tables and merge them